In [15]:
# !git config --global user.name "mrrhbr"
# !git config --global user.email "morvarid.rahbar@gmail.com"

In [14]:
# !ssh-keygen -t rsa -b 4096 -C "mrahbar.2001@gmail.com" -f /root/.ssh/id_rsa -N ""

In [12]:
# !cat /root/.ssh/id_rsa.pub

In [13]:
# !ssh-keyscan github.com >> /root/.ssh/known_hosts

In [1]:
# !pip install kaggle
# !pip install kagglehub
# !pip install timm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [4]:
# from google.colab import files
# files.upload()

In [5]:
# !kaggle datasets list -s chexpert

In [6]:
# !mkdir -p /content/drive/MyDrive/chexpert_data

In [7]:
# %cd /content/drive/MyDrive/
# !kaggle datasets download -d ashery/chexpert -p chexpert_data